# Testing Apache Spark queries 

Disclaimer: O intuito deste trabalho é meramente aprender a utilizar o Apache Spark. Embora os dados utilizados tenham teor político, não há intenção de fazer qualquer tipo apologia, seja positiva ou negativa, a qualquer partido ou candidato.

O autor do trabalho declara não ter vínculo algum com nenhum partido político ou candidato.

Com intuito de negar quaisquer questionamentos, tentarei usar algumas estratégias:
- Sempre utilizar funções aleatórias para printar os dados
- Tentar trabalhar o problema de forma geral, sem focar em nenhum partido ou candidato
- Evitar apresentar nomes de candidatos ao menos que seja necessário

## Instantiating Spark session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    IntegerType, 
    DateType,
    FloatType
)

In [2]:
spark = SparkSession.builder.appName("Test_queries").getOrCreate()

/home/joaopedro/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## Loading csv files from the folder

Bens declarados dos Candidatos: https://dados.gov.br/dataset/candidatos-2022

Brasil states: https://github.com/fititnt/gis-dataset-brasil

### Loading bens

In [3]:
df_bens_raw = (
    spark
    .read
    .option("header", "true")
    .option("encoding", 'latin1')
    .option("delimiter", ";")
    .option("inferSchema", "true")
    .csv('./bem_candidato_2022/bem_candidato_2022_BRASIL.csv')
)

Filtrando as colunas usadas e transformando o valor do bem em float

In [4]:
df_bens = (
    df_bens_raw
    .select(
        [
            "SQ_CANDIDATO",
            "DS_TIPO_BEM_CANDIDATO",
            "DS_BEM_CANDIDATO",
            "VR_BEM_CANDIDATO"
        ]
    )
    .withColumn(
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.')
        .cast(FloatType())
    )
)

In [5]:
df_bens.show(5)

+------------+---------------------+--------------------+----------------+
|SQ_CANDIDATO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|
+------------+---------------------+--------------------+----------------+
|110001608768| Veículo automotor...|               Carro|         40000.0|
|200001608811| Fundos: Ações, Mú...|5.617,50 AÇÕES DA...|        50164.28|
|240001614377|              Terreno|TERRENO URBANO,  ...|        186200.0|
|240001614377| Caderneta de poup...|CADERNETA DE POUP...|         2367.39|
|210001647159| Depósito bancário...|SALDO APLICACAO F...|        22713.17|
+------------+---------------------+--------------------+----------------+
only showing top 5 rows



### Loading Candidatos

In [6]:
df_candidatos_raw = (
    spark
    .read
    .option("header", "true")
    .option("encoding", 'latin1')
    .option("delimiter", ";")
    .option("inferSchema", "true")
    .csv('./consulta_cand_2022/consulta_cand_2022_BRASIL.csv')
)

In [7]:
df_deputados_fed = (
    df_candidatos_raw
    .filter(df_candidatos_raw['DS_SITUACAO_CANDIDATURA'] != 'INAPTO')
    .filter(df_candidatos_raw['CD_CARGO'] == '6') # Código do cargo de deputado federal
    .select(
        [
            "SG_UE",
            "NM_UE",
            "SQ_CANDIDATO",
            "NM_CANDIDATO",
            "SG_PARTIDO",
        ]
    )
)

Visualizando os dados ...

In [8]:
(
    df_deputados_fed
    .select( ["SG_UE", "SG_PARTIDO"] )
    .sample(False, 0.01)
    .show(10)
)

+-----+------------+
|SG_UE|  SG_PARTIDO|
+-----+------------+
|   SC|         PSB|
|   SP|        PRTB|
|   MA|          DC|
|   MG|         MDB|
|   RN|         PSD|
|   PE|         PDT|
|   SP|REPUBLICANOS|
|   SP|         PTB|
|   RJ|          PP|
|   RJ|        AGIR|
+-----+------------+
only showing top 10 rows



## Realizando o Join entre os dois DataFrames

Unir as duas tabelas usando o campo `SQ_CANDIDATO` como chave

In [9]:
df_candidatos_bens = (
    df_deputados_fed
    .join(
        df_bens, 
        on='SQ_CANDIDATO', 
        how='left'
    )
)

In [10]:
df_candidatos_bens.sample(False, 0.01).show(5)

+------------+-----+--------------+--------------------+------------+---------------------+--------------------+----------------+
|SQ_CANDIDATO|SG_UE|         NM_UE|        NM_CANDIDATO|  SG_PARTIDO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|
+------------+-----+--------------+--------------------+------------+---------------------+--------------------+----------------+
|160001654829|   PR|        PARANÁ|EDUARDO PINHEIRO ...|        PODE| Caderneta de poup...|POUPANÇA NO BANCO...|          4000.0|
|160001654829|   PR|        PARANÁ|EDUARDO PINHEIRO ...|        PODE| Veículo automotor...|VEICULO PALIO FIR...|         25000.0|
|190001609750|   RJ|RIO DE JANEIRO|DERCILEI FRANÇA L...|      AVANTE| Quotas ou quinhõe...|empresa Ki Delíci...|             0.0|
|160001621906|   PR|        PARANÁ|ANA FÁBIA RIBAS D...|        PROS| Ações (inclusive ...|40 ACOES ON DO MA...|          1280.0|
|130001619746|   MG|  MINAS GERAIS|BRAZ FERNANDO DA ...|REPUBLICANOS|                 Loja

# Queries

Existem duas formas principais de realizar queries no apache Spark aqui no Python: Funcional e via SQL.
Essa diferença é meramente de sintaxe e escolha pessoal, pois o Spark executa as queries da mesma forma.

Como eu tenho mais familiaridade com o Pandas, para mim, é mais natural utilizar o encadeamento de funções.

**Top 10 candidatos a Deputado Federal com maior patrimônio**

In [17]:
# Nomes e patrimonio dos Candidatos mais ricos
patrimonio_candidatos = (
    df_candidatos_bens
    .groupBy("SQ_CANDIDATO")
    .agg(
        F.sum("VR_BEM_CANDIDATO").alias("VR_BEM_CANDIDATO")
    )
    .join(
        df_deputados_fed,
        on='SQ_CANDIDATO',
        how='left'
    )
    .orderBy(
        F.desc("VR_BEM_CANDIDATO")
    )
    .select(
        [
            "NM_CANDIDATO",
            "SG_PARTIDO",
            "SG_UE",
            "VR_BEM_CANDIDATO",
        ]
    )
    .withColumn(
        "VR_BEM_CANDIDATO",
        F.format_number(
            F.col("VR_BEM_CANDIDATO") / 1e6, 2
        )
    )
)

patrimonio_candidatos.show(10, truncate=False)

+----------------------------------+----------+-----+----------------+
|NM_CANDIDATO                      |SG_PARTIDO|SG_UE|VR_BEM_CANDIDATO|
+----------------------------------+----------+-----+----------------+
|EUNÍCIO LOPES DE OLIVEIRA         |MDB       |CE   |158.18          |
|RUY ADRIANO BORGES MUNIZ          |AVANTE    |MG   |158.02          |
|JOSE GOMES FERREIRA FILHO         |PP        |DF   |128.64          |
|JADYEL SILVA ALENCAR              |PV        |PI   |107.55          |
|PABLO HENRIQUE COSTA MARÇAL       |PROS      |SP   |88.44           |
|ALEX DOS SANTOS GARCIA            |PSC       |RN   |80.01           |
|ADRIANA MANGABEIRA WANDERLEY      |PSD       |DF   |77.06           |
|NEWTON BONIN                      |UNIÃO     |PR   |76.12           |
|HERCILIO ARAUJO DINIZ FILHO       |MDB       |MG   |65.90           |
|PAULO ROBERTO ROQUE ANTONIO KHOURI|NOVO      |DF   |65.29           |
+----------------------------------+----------+-----+----------------+
only s

**Valor total declarado**

In [24]:
(
    df_candidatos_bens
    .agg(
        F.sum("VR_BEM_CANDIDATO").alias("TOTAL_BEM_CANDIDATO_BILHOES"),
        F.countDistinct("SQ_CANDIDATO").alias("TOTAL_CANDIDATOS")
    )
    .withColumn(
        "TOTAL_BEM_CANDIDATO_BILHOES",
        F.format_number(
            F.col("TOTAL_BEM_CANDIDATO_BILHOES") / 1e9, 2
        )
    )
    .show()
)

+---------------------------+----------------+
|TOTAL_BEM_CANDIDATO_BILHOES|TOTAL_CANDIDATOS|
+---------------------------+----------------+
|                       7.21|            9823|
+---------------------------+----------------+



**Valor total declarado de bens por UF**

In [12]:
valor_por_uf = (
    df_candidatos_bens
    .groupBy(
        "SG_UE"
    )
    .agg(
        F.sum( F.col("VR_BEM_CANDIDATO") ).alias("TOTAL BENS")
    )
    .orderBy(
        F.desc("TOTAL BENS")
    )
    .toPandas()
)

In [13]:
valor_por_uf

,SG_UE,TOTAL BENS
0,SP,1.214328e+09
1,MG,9.504192e+08
2,RJ,5.998792e+08
3,PR,4.687365e+08
4,DF,4.338724e+08
5,GO,3.126668e+08
6,CE,2.976541e+08
7,BA,2.840690e+08
8,RS,2.650317e+08
9,SC,2.417713e+08


**Valor total declarado de bens por Tipo de bem**

In [31]:
from pyspark.sql.window import Window

In [32]:
valor_por_tipo_bem = (
    df_candidatos_bens
    .groupBy(
        "DS_TIPO_BEM_CANDIDATO"
    )
    .agg(
        F.sum( F.col("VR_BEM_CANDIDATO") ).alias("TOTAL BENS")
    )
    .orderBy(
        F.desc("TOTAL BENS")
    )
    .withColumn(
        "Percentual",
        # Trick to get the percentage of each row
        # https://stackoverflow.com/questions/48915834/how-to-calculate-percentage-of-each-row-in-pyspark
        F.col("TOTAL BENS") / F.sum(F.col("TOTAL BENS")).over(Window.partitionBy()) * 100
    )
    .toPandas()
)   

In [33]:
valor_por_tipo_bem

,DS_TIPO_BEM_CANDIDATO,TOTAL BENS,Percentual
0,Casa,1.058904e+09,14.695832
1,Quotas ou quinhões de capital,9.503186e+08,13.188846
2,Apartamento,8.900710e+08,12.352709
3,Outros bens imóveis,6.791680e+08,9.425725
4,OUTROS BENS E DIREITOS,5.823553e+08,8.082125
5,Terreno,5.213774e+08,7.235853
6,Outras participações societárias,4.374761e+08,6.071442
7,"Veículo automotor terrestre: caminhão, automóv...",3.962000e+08,5.498598
8,"Aplicação de renda fixa (CDB, RDB e outros)",2.350587e+08,3.262225
9,Crédito decorrente de empréstimo,1.884130e+08,2.614860
